In [4]:
import rdkit
from rdkit.Chem.inchi import *
from rdkit import Chem
from rdkit.Chem.rdchem import Mol

import pandas as pd
from tqdm.notebook import tqdm
import urllib.request
import pubchempy as pcp
tqdm.pandas()

In [5]:
#downloads list of isomers from pubchem
#removes "." containing smiles
#removes duplicates

def find_bad_str(string):
    return string.find('.')==-1
     
def get_list_of_isomers(formula, output_path):
    cmpds=pcp.get_cids(formula, 'formula')
    pcp.download('CSV', output_path, cmpds, operation='property/InChi', overwrite=True)
    data=pd.read_csv(output_path, sep=',')
    data['mol']=data['InChI'].progress_apply(MolFromInchi)
    data['mol']=data['mol'].progress_apply(Chem.RemoveHs)
    data['smiles']=data['mol'].progress_apply(rdkit.Chem.MolToSmiles, isomericSmiles=False)
    data['OK']=data['smiles'].progress_apply(find_bad_str)
    data=data[data['OK']==True]
    data.drop_duplicates(subset ="smiles", inplace = True)
    data.to_csv(output_path, sep='\t', columns=['CID', 'smiles'])
    print (len(data))

In [6]:
get_list_of_isomers('C9H11NO3', 'Hydroxyphenylalanine.csv')


4927
